# Oracle Graph

Oracle Graph is a feature of the graph database developed by Oracle. 
It is integrated as a component of Oracle Database. The feature of Oracle Graph is that it stores graph data as relational data. And graphs are created from them. 
The data elements in Oracle Graph are Vertices and the Relations that connect them, each with Labels and properties.


This notebook explains the basic method for constructing a knowledge graph from unstructured text using an Large Language Model (LLM). 
Additionally, it also introduces how to query the graph.

### Install and Setup

To use this integration, you need to install langchain-community by running `pip install -qU langchain-community`.

To use Oracle Graph with LangChain, install the Oracle Python client driver and the Oracle Property Graph driver as well.

In [ ]:
# pip install oracledb
# pip install opg4py

This is a sample code that demonstrates how to connect to Oracle database

In [ ]:
import oracledb
username = "username"
password = "password"
dsn = "dsn"

try:
    connection = oracledb.connect(user=username, password=password, dsn=dsn)
    print("Connection successful!")
except Exception as e:
    print("Connection failed!")

Next, we will define Oracle Database connection. Follow these steps to set up Oracle Graph.

In [ ]:
from langchain_community.graphs.oracle_graph import OracleGraph

jdbc_url = "jdbc_url"
oraclegraph = OracleGraph(url=jdbc_url, username=username, password=password)
oraclegraph.set_connection(client=connection)

We default to OpenAI models in this guide. 
We define ChatOpenAI as the LLM.

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(api_key=OPENAI_API_KEY, model="gpt-4o")

### LLM Graph Transformer
Now we use `LLMGraphTransformer` to extract graph data from the text.
We can extract categorize entities and relationships from text using LLM.

In [ ]:
from langchain_experimental.graph_transformers import LLMGraphTransformer

llm_transformer = LLMGraphTransformer(
    llm=llm,
    allowed_relationships=["friend", "live_in", "grandchild"],
    node_properties=["person_info"]
    )

In [ ]:
from langchain_core.documents import Document

text = """
Marie Curie, born in 1867, was a Polish and naturalised-French physicist and chemist who conducted pioneering research on radioactivity.
She was the first woman to win a Nobel Prize, the first person to win a Nobel Prize twice, and the only person to win a Nobel Prize in two scientific fields.
Her husband, Pierre Curie, was a co-winner of her first Nobel Prize, making them the first-ever married couple to win the Nobel Prize and launching the Curie family legacy of five Nobel Prizes.
She was, in 1906, the first woman to become a professor at the University of Paris.
"""
documents = [Document(page_content=text)]
graph_documents = llm_transformer.convert_to_graph_documents(documents)


### Storing graph to Oracle Database
From the obtained structured data, we extract information on Oracle Graph's Vertex, Relation, Label, and Property, and create Tables and Graphs within Oracle Database.

In [ ]:
graph_name = 'graph_name'
oraclegraph.add_graph_documents(graph_documents=graph_documents, graph_name=graph_name)

### SQL Query Oracle Graph
One of the features of Oracle Graph is that we can query the graph using Property Graph Query Language (PGQL) and SQL. 
We can query using SQL as follows.

In [ ]:
select_query = """
        SELECT a, b
        FROM GRAPH_TABLE(PERSON_GRAPH
        MATCH (a is Person) - [e] -> (b is Person)
        WHERE e.name = 'FRIEND'
        COLUMNS(a.id as a, b.id as b))
        """
sql_results = oraclegraph.sql_execute(query=select_query)
print(sql_results)